<a href="https://colab.research.google.com/github/chisoftltd/FlightFinder/blob/main/FlightFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twilio

In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)



In [ ]:
!ls /content/drive/MyDrive/100Days_of_CodeTheCompletePythonProBootcamp_for2023/Flight_Deal_Finder
%cd /content/drive/MyDrive/100Days_of_CodeTheCompletePythonProBootcamp_for2023/Flight_Deal_Finder

In [ ]:
class FlightData:

    def __init__(self, price, origin_city, origin_airport, destination_city, destination_airport, out_date, return_date):
        self.price = price
        self.origin_city = origin_city
        self.origin_airport = origin_airport
        self.destination_city = destination_city
        self.destination_airport = destination_airport
        self.out_date = out_date
        self.return_date = return_date

In [ ]:
import requests
SHEETY_PRICES_ENDPOINT = 'https://api.sheety.co/0b454e5fe21e86cfcf0bdab83e13b8bd/benFlightDeals/prices'


class DataManager:

    def __init__(self):
        self.destination_data = {}

    def get_destination_data(self):
        # 2. Use the Sheety API to GET all the data in that sheet and print it out.
        response = requests.get(url=SHEETY_PRICES_ENDPOINT)
        data = response.json()
        self.destination_data = data["prices"]
        # 3. Try importing pretty print and printing the data out again using pprint() to see it formatted.
        # pprint(data)
        return self.destination_data

    # 6. In the DataManager Class make a PUT request and use the row id from sheet_data
    # to update the Google Sheet with the IATA codes. (Do this using code).
    def update_destination_codes(self):
        for city in self.destination_data:
            new_data = {
                "price": {
                    "iataCode": city["iataCode"]
                }
            }
            response = requests.put(
                url=f"{SHEETY_PRICES_ENDPOINT}/{city['id']}",
                json=new_data
            )
            print(response.text)


In [ ]:
TEQUILA_ENDPOINT = "https://tequila-api.kiwi.com"
TEQUILA_API_KEY = "4HE8rB46chl9cNHK64bAN4fsxK45DJNf0369147"


class FlightSearch:

    def get_destination_code(self, city_name):
        location_endpoint = f"{TEQUILA_ENDPOINT}/locations/query"
        headers = {"apikey": TEQUILA_API_KEY}
        query = {"term": city_name, "location_types": "city"}
        response = requests.get(url=location_endpoint, headers=headers, params=query)
        results = response.json()["locations"]
        code = results[0]["code"]
        return code


    def check_flights(self, origin_city_code, destination_city_code, from_time, to_time):
        headers = {"apikey": TEQUILA_API_KEY}
        query = {
            "fly_from": origin_city_code,
            "fly_to": destination_city_code,
            "date_from": from_time.strftime("%d/%m/%Y"),
            "date_to": to_time.strftime("%d/%m/%Y"),
            "nights_in_dst_from": 7,
            "nights_in_dst_to": 28,
            "one_for_city": 1,
            "max_stopovers": 0,
            "curr": "GBP"
        }

        response = requests.get(
            url=f"{TEQUILA_ENDPOINT}/search",
            headers=headers,
            params=query,
        )

        try:
            print(f'FlightSearch.check_flights: {response = }')
            print(f'FlightSearch.check_flights: {response.json() = }')
            data = response.json()["data"][0]
            print(f'FlightSearch.check_flights: {data = }')
        except IndexError:
            print(f"No flights found for {destination_city_code}.")
            return None

        flight_data = FlightData(
            price=data["price"],
            origin_city=data["route"][0]["cityFrom"],
            origin_airport=data["route"][0]["flyFrom"],
            destination_city=data["route"][0]["cityTo"],
            destination_airport=data["route"][0]["flyTo"],
            out_date=data["route"][0]["local_departure"].split("T")[0],
            return_date=data["route"][1]["local_departure"].split("T")[0]
        )
        print(f"{flight_data.destination_city}: £{flight_data.price}")
        return flight_data

In [ ]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os

from twilio.rest import Client

TWILIO_SID = 'AC35908559e4b634d2c2eb2339747e67f8225971'
TWILIO_AUTH_TOKEN = '94fe5d6611325fd1c1ba717352d2ba5b23654'
TWILIO_VIRTUAL_NUMBER = '+234770014244009887'
TWILIO_VERIFIED_NUMBER = '+234759704436412584'


class NotificationManager:

    def __init__(self):
        self.client = Client(TWILIO_SID, TWILIO_AUTH_TOKEN)

    def send_sms(self, message):
        message = self.client.messages.create(
            body=message,
            from_=TWILIO_VIRTUAL_NUMBER,
            to=TWILIO_VERIFIED_NUMBER,
        )
        # Prints if successfully sent.
        print(message.sid)

In [ ]:
# 4. Pass the data back to the main.py file, so that you can print the data from main.py
import sys
sys.path.insert(0, '/content/drive/MyDrive/100Days_of_CodeTheCompletePythonProBootcamp_for2023/Flight_Deal_Finder/data_manager.py')  # Replace with the actual path


from datetime import datetime, timedelta
# from data_manager import DataManager
data_manager = DataManager()
sheet_data = data_manager.get_destination_data()
flight_search = FlightSearch()
# print(sheet_data)

ORIGIN_CITY_IATA = "LON"


#  5. In main.py check if sheet_data contains any values for the "iataCode" key.
#  If not, then the IATA Codes column is empty in the Google Sheet.
#  In this case, pass each city name in sheet_data one-by-one
#  to the FlightSearch class to get the corresponding IATA code
#  for that city using the Flight Search API.
#  You should use the code you get back to update the sheet_data dictionary.
if sheet_data[0]["iataCode"] == "":
    for row in sheet_data:
        row["iataCode"] = flight_search.get_destination_code(row["city"])
    data_manager.destination_data = sheet_data
    data_manager.update_destination_codes()

tomorrow = datetime.now() + timedelta(days=1)
six_month_from_today = datetime.now() + timedelta(days=(6 * 30))



for destination in sheet_data:
    flight = flight_search.check_flights(
        ORIGIN_CITY_IATA,
        destination["iataCode"],
        from_time=tomorrow,
        to_time=six_month_from_today
    )


if flight.price < destination["lowestPrice"]:
        notification_manager.send_sms(
            message=f"Low price alert! Only £{flight.price} to fly from {flight.origin_city}-{flight.origin_airport} to {flight.destination_city}-{flight.destination_airport}, from {flight.out_date} to {flight.return_date}."
        )